# flights data analysis (all preprocessing steps)

# import packages, read original data and pick a subset of columns

In [2]:
# import base packages
# import modin.pandas as pd

import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

# import ray
# # Modin will connect to the existing Ray environment
# import modin.pandas as pd


In [3]:
master_data = pd.read_csv('flights.csv', low_memory=False)

In [4]:
master_data.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [5]:
master_data.weather_delay.unique()

array([  nan,    0.,   25., ..., 1256., 1331., 1306.])

In [6]:
master_data.groupby('origin')['weather_delay'].mean().sort_values().tail()

origin
IMT    26.456853
DRT    28.705882
PIR    29.524752
EKO    29.698630
APN    39.904255
Name: weather_delay, dtype: float64

In [7]:
master_data.groupby('origin')['carrier_delay'].mean().sort_values().tail()

origin
ATY    54.505495
CYS    54.690476
JMS    58.299363
SHD    64.247863
DVL    76.062500
Name: carrier_delay, dtype: float64

In [8]:
master_data.groupby('origin')['security_delay'].mean().sort_values().tail()

origin
DLG     2.923077
OME     3.282258
BET     7.101382
AKN    19.166667
ADK    28.093750
Name: security_delay, dtype: float64

In [9]:
master_data.groupby('origin')['nas_delay'].mean().sort_values().tail()

origin
ACY    37.753309
RDD    39.344765
HOB    41.676329
HYA    46.425532
PQI    59.389961
Name: nas_delay, dtype: float64

In [10]:
master_data.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [11]:
master_data.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-11-30,AA,AA,AA,2367,AA,N957NN,2367,14869,SLC,"Salt Lake City, UT",11298,DFW,"Dallas/Fort Worth, TX",1159,1159.0,0.0,11.0,1210.0,1504.0,6.0,1535,1510.0,-25.0,0,NaN,0,N,156.0,131.0,114.0,1,989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-11-30,AA,AA,AA,2368,AA,N898NN,2368,10821,BWI,"Baltimore, MD",11298,DFW,"Dallas/Fort Worth, TX",1525,1520.0,-5.0,19.0,1539.0,1752.0,8.0,1803,1800.0,-3.0,0,NaN,0,N,218.0,220.0,193.0,1,1217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-11-30,AA,AA,AA,2368,AA,N898NN,2368,11298,DFW,"Dallas/Fort Worth, TX",10821,BWI,"Baltimore, MD",1045,1044.0,-1.0,16.0,1100.0,1425.0,7.0,1438,1432.0,-6.0,0,NaN,0,N,173.0,168.0,145.0,1,1217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-11-30,AA,AA,AA,2371,AA,N771XF,2371,11618,EWR,"Newark, NJ",14107,PHX,"Phoenix, AZ",929,922.0,-7.0,32.0,954.0,1322.0,8.0,1320,1330.0,10.0,0,NaN,0,N,351.0,368.0,328.0,1,2133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-11-30,AA,AA,AA,2372,AA,N883NN,2372,11298,DFW,"Dallas/Fort Worth, TX",14683,SAT,"San Antonio, TX",1748,1745.0,-3.0,16.0,1801.0,1846.0,9.0,1858,1855.0,-3.0,0,NaN,0,N,70.0,70.0,45.0,1,247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# read flights data (note that only columns in flights_test were picked)

data = pd.read_csv('flights.csv', usecols=['arr_delay', 'arr_time', 'crs_arr_time', 'crs_dep_time', 'dep_delay', 'dep_time', 'dest', 'dest_airport_id', 'fl_date', 'origin', 'origin_airport_id', 'tail_num', 'op_unique_carrier', 
                                          'op_carrier_fl_num', 'distance', 'cancellation_code', 'air_time'], low_memory=False)  

In [13]:
data.shape

(15927485, 17)

In [14]:
# create a dictionary to read unique counts in each column

{column: len(data[column].unique()) for column in data.columns}

{'fl_date': 730,
 'op_unique_carrier': 28,
 'tail_num': 6488,
 'op_carrier_fl_num': 7252,
 'origin_airport_id': 376,
 'origin': 376,
 'dest_airport_id': 376,
 'dest': 376,
 'crs_dep_time': 1412,
 'dep_time': 1441,
 'dep_delay': 1750,
 'crs_arr_time': 1437,
 'arr_time': 1441,
 'arr_delay': 1782,
 'cancellation_code': 6,
 'air_time': 698,
 'distance': 1623}

In [15]:
data.head()

,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,dest_airport_id,dest,crs_dep_time,dep_time,dep_delay,crs_arr_time,arr_time,arr_delay,cancellation_code,air_time,distance
0,2019-11-30,AA,N957NN,2367,14869,SLC,11298,DFW,1159,1159.0,0.0,1535,1510.0,-25.0,NaN,114.0,989
1,2019-11-30,AA,N898NN,2368,10821,BWI,11298,DFW,1525,1520.0,-5.0,1803,1800.0,-3.0,NaN,193.0,1217
2,2019-11-30,AA,N898NN,2368,11298,DFW,10821,BWI,1045,1044.0,-1.0,1438,1432.0,-6.0,NaN,145.0,1217
3,2019-11-30,AA,N771XF,2371,11618,EWR,14107,PHX,929,922.0,-7.0,1320,1330.0,10.0,NaN,328.0,2133
4,2019-11-30,AA,N883NN,2372,11298,DFW,14683,SAT,1748,1745.0,-3.0,1858,1855.0,-3.0,NaN,45.0,247


In [16]:
#planes = pd.read_csv('planes.csv')

In [17]:
#planes = planes.rename(columns = {'tailnum':'tail_num'})

In [18]:
#planes = planes[['tail_num', 'model']]

In [19]:
#type(planes)

# sampling flights data

In [20]:
flights_sample = data.sample(frac=1, random_state=143)

In [21]:
#flights_sample = pd.merge(flights_sample, planes, on="tail_num", how='left')

In [22]:
flights_sample.shape

(15927485, 17)

In [23]:
flights_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15927485 entries, 7153908 to 14516814
Data columns (total 17 columns):
 #   Column             Dtype  
---  ------             -----  
 0   fl_date            object 
 1   op_unique_carrier  object 
 2   tail_num           object 
 3   op_carrier_fl_num  int64  
 4   origin_airport_id  int64  
 5   origin             object 
 6   dest_airport_id    int64  
 7   dest               object 
 8   crs_dep_time       int64  
 9   dep_time           float64
 10  dep_delay          float64
 11  crs_arr_time       int64  
 12  arr_time           float64
 13  arr_delay          float64
 14  cancellation_code  object 
 15  air_time           float64
 16  distance           int64  
dtypes: float64(5), int64(6), object(6)
memory usage: 2.1+ GB


In [24]:
# flights_sample.describe()

In [25]:
# check for null values in the selected dataset

flights_sample.isna().mean()

fl_date              0.000000
op_unique_carrier    0.000000
tail_num             0.003093
op_carrier_fl_num    0.000000
origin_airport_id    0.000000
origin               0.000000
dest_airport_id      0.000000
dest                 0.000000
crs_dep_time         0.000000
dep_time             0.016250
dep_delay            0.016560
crs_arr_time         0.000000
arr_time             0.017271
arr_delay            0.019573
cancellation_code    0.983098
air_time             0.019791
distance             0.000000
dtype: float64

# feature pre-processing

In [26]:
# helper function to pre-process data

#import datetime
#import holidays

def preprocess_data(df):
    df = df.copy()
    ## strip date to create new useful features
    df['month'] = pd.to_datetime(df['fl_date']).dt.month # add month column
    df['year'] = pd.to_datetime(df['fl_date']).dt.year # add year column
    df['day_of_week'] = pd.to_datetime(df['fl_date']).dt.dayofweek # add week column. It is assumed the week starts on Monday, which is denoted by 0 and ends on Sunday which is denoted by 6.
    df['day_of_month'] = pd.to_datetime(df['fl_date']).dt.day # add day of month
    
    ## create dep_hour from crs_dep_time
    df['crs_dep_time'] = df.crs_dep_time[~df.crs_dep_time.isna()].astype(np.int64).apply('{:0>4}'.format)
    df['crs_dep_time'] = pd.to_timedelta(df.crs_dep_time.str[:2]+':'+df.crs_dep_time.str[2:]+':00')
    df['dep_hour'] = df.crs_dep_time.dt.components.hours
    
    ## create arr_hour from crs_arr_time    
    df['crs_arr_time'] = df.crs_arr_time[~df.crs_arr_time.isna()].astype(np.int64).apply('{:0>4}'.format)
    df['crs_arr_time'] = pd.to_timedelta(df.crs_arr_time.str[:2]+':'+df.crs_arr_time.str[2:]+':00')
    df['arr_hour'] = df.crs_arr_time.dt.components.hours
    
    # create a column combining op_unique_carrier, origin and destination airports
    df['unique_fl'] = df['op_unique_carrier'].astype(str) + df['origin'].astype(str) + df['dest'].astype(str)
    
    # create a column for speed
    df['speed'] = df['distance']/(df['air_time']/60)
    
    # create a column with ontime, late or cancelled status for departure
    df.loc[df.dep_delay.isna(), 'dep_status']='cancelled'
    df.loc[df.dep_delay <= 0, 'dep_status']='ontime'
    df.loc[df.dep_delay > 0, 'dep_status']='late'
        
    df.loc[df.arr_delay.isna(), 'arr_status']='canceled'
    df.loc[df.arr_delay <= 0, 'arr_status']='ontime'
    df.loc[df.arr_delay > 0, 'arr_status']='late'
         
    # fill NaN values in categorical data columns
    df['cancellation_code'] = df['cancellation_code'].replace(np.nan, 'N') # 'N' technically means no-delay
        
    # fill null values in numerical data columns their mean
    df['speed'] = df['speed'].fillna(df['speed'].mean())
    df['air_time'] = df['air_time'].fillna(df['air_time'].mean())
    df['dep_delay'] = df['dep_delay'].fillna(df['dep_delay'].mean())
    df['arr_delay'] = df['arr_delay'].fillna(df['arr_delay'].mean())
    
    # clip negative values as dep_delay or arrival_delay of negative values is the same meaning as no-delay
    df['dep_delay'] = df['dep_delay'].clip(lower=0)
    df['arr_delay'] = df['arr_delay'].clip(lower=0)
    
  
    # use proper datatypes
    df[['op_unique_carrier', 'origin_airport_id', 'origin', 'dest_airport_id', 'dest', 'dep_status', 'arr_status', 'cancellation_code', 'unique_fl', 'op_carrier_fl_num', 'month', 'day_of_week', 'day_of_month', 'dep_hour', 'arr_hour']] = df[['op_unique_carrier', 'origin_airport_id', 'origin', 'dest_airport_id', 'dest', 'dep_status', 'arr_status', 'cancellation_code', 'unique_fl', 'op_carrier_fl_num', 'month', 'day_of_week', 'day_of_month', 'dep_hour', 'arr_hour']].apply(lambda x: x.astype('category'))
    
    # drop useless columns
    df = df.drop(['fl_date', 'tail_num', 'crs_dep_time', 'crs_arr_time', 'dep_time', 'arr_time', 'year'], axis=1)  
    
    # drop any duplicate rows
    df = df.drop_duplicates()
        
    # encode cyclical features using sin and cosine transformation
    
    return df

In [27]:
# flights_sample.columns

In [28]:
X = preprocess_data(flights_sample) #run the pre-processing steps on flights_sample and pass the df to new variable X

In [29]:
X.head(2)

,op_unique_carrier,op_carrier_fl_num,origin_airport_id,origin,dest_airport_id,dest,dep_delay,arr_delay,cancellation_code,air_time,distance,month,day_of_week,day_of_month,dep_hour,arr_hour,unique_fl,speed,dep_status,arr_status
7153908,WN,628,14679,SAN,14107,PHX,26.0,11.0,N,52.0,304,5,2,29,18,19,WNSANPHX,350.769231,late,late
13164250,OO,5565,11292,DEN,11638,FAT,9.0,0.0,N,106.0,844,3,0,26,19,20,OODENFAT,477.735849,late,ontime


In [30]:
X.info() #check dtypes if all features have correct types

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15927485 entries, 7153908 to 14516814
Data columns (total 20 columns):
 #   Column             Dtype   
---  ------             -----   
 0   op_unique_carrier  category
 1   op_carrier_fl_num  category
 2   origin_airport_id  category
 3   origin             category
 4   dest_airport_id    category
 5   dest               category
 6   dep_delay          float64 
 7   arr_delay          float64 
 8   cancellation_code  category
 9   air_time           float64 
 10  distance           int64   
 11  month              category
 12  day_of_week        category
 13  day_of_month       category
 14  dep_hour           category
 15  arr_hour           category
 16  unique_fl          category
 17  speed              float64 
 18  dep_status         category
 19  arr_status         category
dtypes: category(15), float64(4), int64(1)
memory usage: 1.0 GB


In [31]:
X.isna().mean() #check for the null values | can use sum() as well but mean() gives better insights

op_unique_carrier    0.0
op_carrier_fl_num    0.0
origin_airport_id    0.0
origin               0.0
dest_airport_id      0.0
dest                 0.0
dep_delay            0.0
arr_delay            0.0
cancellation_code    0.0
air_time             0.0
distance             0.0
month                0.0
day_of_week          0.0
day_of_month         0.0
dep_hour             0.0
arr_hour             0.0
unique_fl            0.0
speed                0.0
dep_status           0.0
arr_status           0.0
dtype: float64

In [32]:
{column: len(X[column].unique()) for column in X.columns} #check for unique items in each feature and return counts as a dictionary

{'op_unique_carrier': 28,
 'op_carrier_fl_num': 7252,
 'origin_airport_id': 376,
 'origin': 376,
 'dest_airport_id': 376,
 'dest': 376,
 'dep_delay': 1619,
 'arr_delay': 1616,
 'cancellation_code': 6,
 'air_time': 698,
 'distance': 1623,
 'month': 12,
 'day_of_week': 7,
 'day_of_month': 31,
 'dep_hour': 24,
 'arr_hour': 24,
 'unique_fl': 17136,
 'speed': 91773,
 'dep_status': 3,
 'arr_status': 3}

In [33]:
X.shape

(15927485, 20)

In [34]:
X['cancellation_code'].value_counts()

N    15658277
B      142089
A       70569
C       56484
D          64
X           2
Name: cancellation_code, dtype: int64

In [35]:
X['unique_fl'].value_counts().head()

HAOGGHNL    19620
HAHNLOGG    19614
HAKOAHNL    14166
HAHNLKOA    13926
HAHNLLIH    13549
Name: unique_fl, dtype: int64

# feature transformation

In [36]:
X1 = X.copy()

## remove outliers (using IQR)

In [37]:
# helper function to remove outliers in X (adjusted iqr to be between 0.25 and 0.70) based on arrival delay

def fun(serie):
    return pd.DataFrame([[serie.quantile(0.25), serie.quantile(0.75)]]
                             *serie.shape[0], 
                        columns=['lower', 'upper'], 
                        index=serie.index)

X1[['lower', 'upper']] = X1.groupby('unique_fl')['arr_delay'].apply(fun)
X1 = X1.query('lower <= `arr_delay` <= upper').drop(columns=['lower', 'upper'])

In [38]:
print(X.shape)
print(X1.shape)

(15927485, 20)
(12071082, 20)


In [39]:
X.describe()

,dep_delay,arr_delay,air_time,distance,speed
count,1.592748e+07,1.592748e+07,1.592748e+07,1.592748e+07,1.592748e+07
mean,1.379819e+01,1.387899e+01,1.079393e+02,7.679726e+02,3.989259e+02
std,4.630228e+01,4.600573e+01,6.898247e+01,5.858934e+02,7.365428e+01
min,0.000000e+00,0.000000e+00,-1.384000e+03,1.600000e+01,-4.212632e+02
25%,0.000000e+00,0.000000e+00,5.800000e+01,3.380000e+02,3.547368e+02
50%,0.000000e+00,0.000000e+00,9.000000e+01,6.030000e+02,4.042623e+02
75%,8.000000e+00,7.000000e+00,1.360000e+02,1.005000e+03,4.493333e+02
max,2.976000e+03,2.973000e+03,1.557000e+03,5.095000e+03,3.696000e+03


In [40]:
X1.describe()

,dep_delay,arr_delay,air_time,distance,speed
count,1.207108e+07,1.207108e+07,1.207108e+07,1.207108e+07,1.207108e+07
mean,1.897141e+00,9.307772e-01,1.070145e+02,7.675787e+02,4.018700e+02
std,7.084063e+00,3.105215e+00,6.830826e+01,5.860167e+02,7.274390e+01
min,0.000000e+00,0.000000e+00,-1.244000e+03,1.600000e+01,-1.088525e+02
25%,0.000000e+00,0.000000e+00,5.800000e+01,3.380000e+02,3.584615e+02
50%,0.000000e+00,0.000000e+00,9.000000e+01,6.020000e+02,4.069565e+02
75%,0.000000e+00,0.000000e+00,1.340000e+02,1.001000e+03,4.514286e+02
max,1.921000e+03,1.916000e+03,6.920000e+02,5.095000e+03,3.696000e+03


## check and correct for skew in numeric columns (stats library from scipy)

In [41]:
X2 = X1.copy() #df for checking skew

In [42]:
X2['mean_arr_delay'] = X2.groupby('unique_fl')['arr_delay'].transform('mean') # add another column for median_arr_delay

In [43]:
X2.head()

,op_unique_carrier,op_carrier_fl_num,origin_airport_id,origin,dest_airport_id,dest,dep_delay,arr_delay,cancellation_code,air_time,distance,month,day_of_week,day_of_month,dep_hour,arr_hour,unique_fl,speed,dep_status,arr_status,mean_arr_delay
13164250,OO,5565,11292,DEN,11638,FAT,9.0,0.0,N,106.0,844,3,0,26,19,20,OODENFAT,477.735849,late,ontime,1.013912
6218269,B6,8,12889,LAS,11697,FLL,25.0,4.0,N,252.0,2173,7,2,18,21,5,B6LASFLL,517.380952,late,late,2.619379
12657684,DL,1389,10397,ATL,10994,CHS,0.0,0.0,N,43.0,259,3,4,2,12,13,DLATLCHS,361.395349,ontime,ontime,0.000000
12506608,MQ,4090,14108,PIA,13930,ORD,0.0,0.0,N,35.0,130,6,1,25,6,7,MQPIAORD,222.857143,ontime,ontime,0.620917
6859796,9E,3480,12339,IND,12478,JFK,0.0,0.0,N,99.0,665,5,2,15,6,9,9EINDJFK,403.030303,ontime,ontime,0.513103


In [44]:
X2['mean_arr_delay'] = X2['mean_arr_delay'].clip(lower=0) # remove negative values as they mean no delay anyway

In [45]:
#X2.head(50).sort_values(by='mean_arr_delay', ascending=False)

In [46]:
# check for skew in numeric columns
import scipy.stats

skew_df = pd.DataFrame(X2.select_dtypes(np.number).columns, columns=['feature'])
skew_df['skew'] = skew_df['feature'].apply(lambda feature: scipy.stats.skew(X2[feature]))
skew_df['absolute skew'] = skew_df['skew'].apply(abs)
skew_df['skewed'] = skew_df['absolute skew'].apply(lambda x: True if x >= 0.5 else False)
skew_df

,feature,skew,absolute skew,skewed
0,dep_delay,40.040178,40.040178,True
1,arr_delay,45.309800,45.309800,True
2,air_time,1.529705,1.529705,True
3,distance,1.554059,1.554059,True
4,speed,-0.430253,0.430253,False
5,mean_arr_delay,339.930646,339.930646,True


In [47]:
#X2.info()

In [48]:
X2.head()

,op_unique_carrier,op_carrier_fl_num,origin_airport_id,origin,dest_airport_id,dest,dep_delay,arr_delay,cancellation_code,air_time,distance,month,day_of_week,day_of_month,dep_hour,arr_hour,unique_fl,speed,dep_status,arr_status,mean_arr_delay
13164250,OO,5565,11292,DEN,11638,FAT,9.0,0.0,N,106.0,844,3,0,26,19,20,OODENFAT,477.735849,late,ontime,1.013912
6218269,B6,8,12889,LAS,11697,FLL,25.0,4.0,N,252.0,2173,7,2,18,21,5,B6LASFLL,517.380952,late,late,2.619379
12657684,DL,1389,10397,ATL,10994,CHS,0.0,0.0,N,43.0,259,3,4,2,12,13,DLATLCHS,361.395349,ontime,ontime,0.000000
12506608,MQ,4090,14108,PIA,13930,ORD,0.0,0.0,N,35.0,130,6,1,25,6,7,MQPIAORD,222.857143,ontime,ontime,0.620917
6859796,9E,3480,12339,IND,12478,JFK,0.0,0.0,N,99.0,665,5,2,15,6,9,9EINDJFK,403.030303,ontime,ontime,0.513103


In [49]:
X3 = X2.copy() #df for log transform of skewed columns

In [50]:
# log transform skewed numeric columns

for column in skew_df.query("skewed == True")['feature'].values:
    X3[column] = np.log1p(X3[column])

/home/callingtad/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [51]:
X3.head()

,op_unique_carrier,op_carrier_fl_num,origin_airport_id,origin,dest_airport_id,dest,dep_delay,arr_delay,cancellation_code,air_time,distance,month,day_of_week,day_of_month,dep_hour,arr_hour,unique_fl,speed,dep_status,arr_status,mean_arr_delay
13164250,OO,5565,11292,DEN,11638,FAT,2.302585,0.000000,N,4.672829,6.739337,3,0,26,19,20,OODENFAT,477.735849,late,ontime,0.700079
6218269,B6,8,12889,LAS,11697,FLL,3.258097,1.609438,N,5.533389,7.684324,7,2,18,21,5,B6LASFLL,517.380952,late,late,1.286303
12657684,DL,1389,10397,ATL,10994,CHS,0.000000,0.000000,N,3.784190,5.560682,3,4,2,12,13,DLATLCHS,361.395349,ontime,ontime,0.000000
12506608,MQ,4090,14108,PIA,13930,ORD,0.000000,0.000000,N,3.583519,4.875197,6,1,25,6,7,MQPIAORD,222.857143,ontime,ontime,0.482992
6859796,9E,3480,12339,IND,12478,JFK,0.000000,0.000000,N,4.605170,6.501290,5,2,15,6,9,9EINDJFK,403.030303,ontime,ontime,0.414163


## encoding for cyclical categorical variables

In [52]:
X4 = X3.copy() # make a new copy for cat variables transformation

In [53]:
X4.head()

,op_unique_carrier,op_carrier_fl_num,origin_airport_id,origin,dest_airport_id,dest,dep_delay,arr_delay,cancellation_code,air_time,distance,month,day_of_week,day_of_month,dep_hour,arr_hour,unique_fl,speed,dep_status,arr_status,mean_arr_delay
13164250,OO,5565,11292,DEN,11638,FAT,2.302585,0.000000,N,4.672829,6.739337,3,0,26,19,20,OODENFAT,477.735849,late,ontime,0.700079
6218269,B6,8,12889,LAS,11697,FLL,3.258097,1.609438,N,5.533389,7.684324,7,2,18,21,5,B6LASFLL,517.380952,late,late,1.286303
12657684,DL,1389,10397,ATL,10994,CHS,0.000000,0.000000,N,3.784190,5.560682,3,4,2,12,13,DLATLCHS,361.395349,ontime,ontime,0.000000
12506608,MQ,4090,14108,PIA,13930,ORD,0.000000,0.000000,N,3.583519,4.875197,6,1,25,6,7,MQPIAORD,222.857143,ontime,ontime,0.482992
6859796,9E,3480,12339,IND,12478,JFK,0.000000,0.000000,N,4.605170,6.501290,5,2,15,6,9,9EINDJFK,403.030303,ontime,ontime,0.414163


In [54]:
#X4['dep_hour'].value_counts().sort_values()

In [55]:
# sin-cos transformation for cyclical conditions

X4['month_sin'] = np.sin(2 * np.pi * X4['month'].astype('int')/12.0)
X4['month_cos'] = np.cos(2 * np.pi * X4['month'].astype('int')/12.0)

X4['day_of_week_sin'] = np.sin(2 * np.pi * X4['day_of_week'].astype('int')/6.0)
X4['day_of_week_cos'] = np.cos(2 * np.pi * X4['day_of_week'].astype('int')/6.0)
   

X4['day_of_month_sin'] = np.sin(2 * np.pi * X4['day_of_month'].astype('int')/31.0)
X4['day_of_month_cos'] = np.cos(2 * np.pi * X4['day_of_month'].astype('int')/31.0)
        

X4['dep_hour_sin'] = np.sin(2 * np.pi * X4['dep_hour'].astype('int')/23.0)
X4['dep_hour_cos'] = np.cos(2 * np.pi * X4['dep_hour'].astype('int')/23.0)
         

X4['arr_hour_sin'] = np.sin(2 * np.pi * X4['arr_hour'].astype('int')/23.0)
X4['arr_hour_cos'] = np.cos(2 * np.pi * X4['arr_hour'].astype('int')/23.0)

X4 = X4.drop(['month', 'day_of_week', 'day_of_month', 'dep_hour', 'arr_hour'], axis=1)
       

In [56]:
X4.head()

,op_unique_carrier,op_carrier_fl_num,origin_airport_id,origin,dest_airport_id,dest,dep_delay,arr_delay,cancellation_code,air_time,distance,unique_fl,speed,dep_status,arr_status,mean_arr_delay,month_sin,month_cos,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,dep_hour_sin,dep_hour_cos,arr_hour_sin,arr_hour_cos
13164250,OO,5565,11292,DEN,11638,FAT,2.302585,0.000000,N,4.672829,6.739337,OODENFAT,477.735849,late,ontime,0.700079,1.000000e+00,6.123234e-17,0.000000,1.0,-0.848644,0.528964,-0.887885,0.460065,-0.730836,0.682553
6218269,B6,8,12889,LAS,11697,FLL,3.258097,1.609438,N,5.533389,7.684324,B6LASFLL,517.380952,late,late,1.286303,-5.000000e-01,-8.660254e-01,0.866025,-0.5,-0.485302,-0.874347,-0.519584,0.854419,0.979084,0.203456
12657684,DL,1389,10397,ATL,10994,CHS,0.000000,0.000000,N,3.784190,5.560682,DLATLCHS,361.395349,ontime,ontime,0.000000,1.000000e+00,6.123234e-17,-0.866025,-0.5,0.394356,0.918958,-0.136167,-0.990686,-0.398401,-0.917211
12506608,MQ,4090,14108,PIA,13930,ORD,0.000000,0.000000,N,3.583519,4.875197,MQPIAORD,222.857143,ontime,ontime,0.482992,1.224647e-16,-1.000000e+00,0.866025,0.5,-0.937752,0.347305,0.997669,-0.068242,0.942261,-0.334880
6859796,9E,3480,12339,IND,12478,JFK,0.000000,0.000000,N,4.605170,6.501290,9EINDJFK,403.030303,ontime,ontime,0.414163,5.000000e-01,-8.660254e-01,0.866025,-0.5,0.101168,-0.994869,0.997669,-0.068242,0.631088,-0.775711


## frequency encoding for categorical variables  

In [57]:
X5 = X4.copy()

In [58]:
# frequency encoding for op_unique_carrier, op_carrier_fl_num, origin_airport_id, origin, dest_airport_id, dest, cancellation_code, unique_fl, dep_status, arr_status 

#, 'op_carrier_fl_num', 'origin_airport_id', 'origin', 'dest_airport_id', 'dest', 'cancellation_code', 'unique_fl', 'dep_status', 'arr_status'

# list = ['op_unique_carrier']

# def catEncode(df):
#     for item in list:
#         encoding = df[item].value_counts(normalize=True)
#         df[item] = df[item].apply(lambda x: encoding[x])
#         return df

# catEncode(X4)        

enc_op_unique_carrier = X5['op_unique_carrier'].value_counts() #(normalize=True)
X5['op_unique_carrier'] = X5['op_unique_carrier'].apply(lambda x : enc_op_unique_carrier[x])

# enc_op_carrier_fl_num = X5['op_carrier_fl_num'].value_counts(normalize=True)
# X5['op_carrier_fl_num'] = X5['op_carrier_fl_num'].apply(lambda x : enc_op_carrier_fl_num[x])

# enc_origin_airport_id = X5['origin_airport_id'].value_counts(normalize=True)
# X5['origin_airport_id'] = X5['origin_airport_id'].apply(lambda x : enc_origin_airport_id[x])

enc_origin = X5['origin'].value_counts() #(normalize=True)
X5['origin'] = X5['origin'].apply(lambda x : enc_origin[x])

# enc_dest_airport_id = X5['dest_airport_id'].value_counts(normalize=True)
# X5['dest_airport_id'] = X5['dest_airport_id'].apply(lambda x : enc_dest_airport_id[x])

enc_dest = X5['dest'].value_counts() #(normalize=True)
X5['dest'] = X5['dest'].apply(lambda x : enc_dest[x])

enc_cancellation_code = X5['cancellation_code'].value_counts() #(normalize=True)
X5['cancellation_code'] = X5['cancellation_code'].apply(lambda x : enc_cancellation_code[x])

enc_unique_fl = X5['unique_fl'].value_counts() #(normalize=True)
X5['unique_fl'] = X5['unique_fl'].apply(lambda x : enc_unique_fl[x])

enc_dep_status = X5['dep_status'].value_counts() #(normalize=True)
X5['dep_status'] = X5['dep_status'].apply(lambda x : enc_dep_status[x])

enc_arr_status = X5['arr_status'].value_counts() #(normalize=True)
X5['arr_status'] = X5['arr_status'].apply(lambda x : enc_arr_status[x])

X5 = X5.drop(['op_carrier_fl_num', 'origin_airport_id', 'dest_airport_id'], axis=1)

In [59]:
X5.head()

,op_unique_carrier,origin,dest,dep_delay,arr_delay,cancellation_code,air_time,distance,unique_fl,speed,dep_status,arr_status,mean_arr_delay,month_sin,month_cos,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,dep_hour_sin,dep_hour_cos,arr_hour_sin,arr_hour_cos
13164250,1222589,417043,18829,2.302585,0.000000,11836974,4.672829,6.739337,1865,477.735849,2269876,10138922,0.700079,1.000000e+00,6.123234e-17,0.000000,1.0,-0.848644,0.528964,-0.887885,0.460065,-0.730836,0.682553
6218269,452851,251909,147969,3.258097,1.609438,11836974,5.533389,7.684324,568,517.380952,2269876,1666388,1.286303,-5.000000e-01,-8.660254e-01,0.866025,-0.5,-0.485302,-0.874347,-0.519584,0.854419,0.979084,0.203456
12657684,1480677,595684,40093,0.000000,0.000000,11836974,3.784190,5.560682,5391,361.395349,9571927,10138922,0.000000,1.000000e+00,6.123234e-17,-0.866025,-0.5,0.394356,0.918958,-0.136167,-0.990686,-0.398401,-0.917211
12506608,471077,9251,597024,0.000000,0.000000,11836974,3.583519,4.875197,959,222.857143,9571927,10138922,0.482992,1.224647e-16,-1.000000e+00,0.866025,0.5,-0.937752,0.347305,0.997669,-0.068242,0.942261,-0.334880
6859796,381866,78884,194668,0.000000,0.000000,11836974,4.605170,6.501290,1127,403.030303,9571927,10138922,0.414163,5.000000e-01,-8.660254e-01,0.866025,-0.5,0.101168,-0.994869,0.997669,-0.068242,0.631088,-0.775711


In [60]:
#len(X5['unique_fl'].value_counts())

## get_dummies for airport id's

In [61]:
X6 = X5.copy()

In [62]:
# X6 = pd.get_dummies(X6, columns=['origin_airport_id'])
# X6 = pd.get_dummies(X6, columns=['dest_airport_id'])

In [63]:
X6.shape

(12071082, 23)

In [64]:
X6.head()

,op_unique_carrier,origin,dest,dep_delay,arr_delay,cancellation_code,air_time,distance,unique_fl,speed,dep_status,arr_status,mean_arr_delay,month_sin,month_cos,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,dep_hour_sin,dep_hour_cos,arr_hour_sin,arr_hour_cos
13164250,1222589,417043,18829,2.302585,0.000000,11836974,4.672829,6.739337,1865,477.735849,2269876,10138922,0.700079,1.000000e+00,6.123234e-17,0.000000,1.0,-0.848644,0.528964,-0.887885,0.460065,-0.730836,0.682553
6218269,452851,251909,147969,3.258097,1.609438,11836974,5.533389,7.684324,568,517.380952,2269876,1666388,1.286303,-5.000000e-01,-8.660254e-01,0.866025,-0.5,-0.485302,-0.874347,-0.519584,0.854419,0.979084,0.203456
12657684,1480677,595684,40093,0.000000,0.000000,11836974,3.784190,5.560682,5391,361.395349,9571927,10138922,0.000000,1.000000e+00,6.123234e-17,-0.866025,-0.5,0.394356,0.918958,-0.136167,-0.990686,-0.398401,-0.917211
12506608,471077,9251,597024,0.000000,0.000000,11836974,3.583519,4.875197,959,222.857143,9571927,10138922,0.482992,1.224647e-16,-1.000000e+00,0.866025,0.5,-0.937752,0.347305,0.997669,-0.068242,0.942261,-0.334880
6859796,381866,78884,194668,0.000000,0.000000,11836974,4.605170,6.501290,1127,403.030303,9571927,10138922,0.414163,5.000000e-01,-8.660254e-01,0.866025,-0.5,0.101168,-0.994869,0.997669,-0.068242,0.631088,-0.775711
